# How Runnables Work in LangChain

## Overview

In LangChain, a runnable can be any Python callable, such as a function, a lambda expression, or an instance method of a class. However, instead of directly passing these callables around, you wrap them in a runnable object to provide additional functionality and metadata, like the function name, execution time, or custom annotations.

Here's an example of how you can create a runnable from a function:

In [1]:
from langchain_core.runnables import RunnableLambda


# Define a simple function
def greet(name):
   return f"Hello, {name}!"


# Wrap the function in a RunnableWrapper
greet_runnable = RunnableLambda(lambda x: greet(x))


# Use the runnable to call the function
result = greet_runnable.invoke("Rama")
print(result)  # Output: Hello, Rama!

Hello, Rama!


In the above code, we defined a simple `greet` function that takes a name as an argument and returns a greeting string. This function is then wrapped in a `RunnableWrapper`. 

`greet_runnable` provides additional functionality and metadata, making it easier to integrate with other parts of your code. This allows you to manage and pass around multiple callables with additional context or behavior. 

One advantage of wrapping callables as runnables is you can now connect them using LangChain's chaining mechanisms, such as the pipe operator (`|`), the `RunnableSequence` class, or the . pipe( ) method.

For example, you can use `RunnableSequence` to create a chain applying multiple transformations to some input data:

In [2]:
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnableSequence

# Define the transformations as simple functions
def greet(name):
   return f"Hello, {name}!"


def append_datetime(text):
   current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
   return f"{text} The current date and time is {current_datetime}."


def to_uppercase(text):
   return text.upper()


def add_exclamation(text):
   return f"{text}!"


# Wrap the functions in RunnableWrapper
greet_runnable = RunnableLambda(lambda x: greet(x))
datetime_runnable = RunnableLambda(lambda x: append_datetime(x))
uppercase_runnable = RunnableLambda(lambda x: to_uppercase(x))
exclamation_runnable = RunnableLambda(lambda x: add_exclamation(x))


# Create a RunnableSequence with the wrapped runnables
chain = RunnableSequence(
   first=greet_runnable,
   middle=[datetime_runnable, uppercase_runnable],
   last=exclamation_runnable,
)


# Apply the chain to some input data
input_data = "Rama"
result = chain.invoke(input_data)
print(
   result
)  # Output example: "HELLO, ALICE! THE CURRENT DATE AND TIME IS 2024-06-19 14:30:00!"

HELLO, RAMA! THE CURRENT DATE AND TIME IS 2024-09-14 15:53:45.!


Here we have four simple functions: `greet`, `append_datetime`, `to_uppercase`, and `add_exclamation`, each of which takes input and performs a specific transformation on it. `RunnableLambda` takes a function as its argument, and creates a runnable object.

We can then create a `RunnableSequence` by passing these runnables to its constructor:

In [3]:
chain = RunnableSequence(
   first=greet_runnable,
   middle=[datetime_runnable, uppercase_runnable],
   last=exclamation_runnable,
)

`RunnableSequence` executes these runnables in sequential order, using the output of one runnable as input to the next.

The result of a chain is a `RunnableSequence` which is still a runnable that can still be piped, invoked, streamed, etc.

## Creating A Runnable with the Chain Decorator
The `@chain` decorator allows you to turn any function into a chain. Below, the decorator creates a custom chain that combines multiple components, such as prompts, models, and output parsers, and defines a function (`custom_chain`) that encapsulates the sequence of operations:

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

@chain
def custom_chain(text):
    prompt_val1 = prompt1.invoke({"topic": text})
    output1 = ChatOpenAI().invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    chain2 = prompt2 | ChatOpenAI() | StrOutputParser()
    return chain2.invoke({"joke": parsed_output1})

`invoke`, `batch`, and `stream` Methods
As previously mentioned, LangChain runnables provide three key methods to execute and interact with your chains:

`invoke`: executes a runnable with a single input, and is typically used when you have a single piece of data to process.
‍
`batch`: allows you to process multiple inputs in parallel. This method is useful when you have a list of inputs and want to run them through the chain simultaneously.
‍
`stream`: processes input data as a stream, handling one piece of data at a time and providing results as they are available. This method is ideal for handling streamed output for real-time data processing or for large datasets that you want to process incrementally. At the time of this writing, streaming support for retries is being added for higher reliability without any latency cost (as explained in their docs).

## Key Runnable Types in LangChain

Within LangChain, you have access to various runnable types that allow you to execute and manage tasks:

`RunnableParallel` for parallelizing operations.
‍
`RunnablePassthrough` for passing data unchanged from previous steps for use as input in later steps.    
‍
`RunnableLambda` for converting a Python callable into a runnable.

## RunnableParallel
This runs a mapping of runnables in parallel and returns a mapping of their outputs. It’s essentially a dictionary whose values are runnables, and it invokes them concurrently, providing the same input to each. 

A `RunnableParallel` can be instantiated directly or by using a dictionary literal within a sequence. This is particularly useful when you want to parallelize operations or manipulate the output of one runnable to match the input format of the next runnable in a sequence.

Below is an example that uses functions to illustrate how `RunnableParallel` works.

In [7]:
import asyncio

from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
   return x + 1

def mul_two(x: int) -> int:
   return x * 2

def mul_three(x: int) -> int:
   return x * 3

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
runnable_3 = RunnableLambda(mul_three)


sequence = runnable_1 | {  # this dict is coerced to a RunnableParallel
   "mul_two": runnable_2,
   "mul_three": runnable_3,
}
# Or equivalently:
# sequence = runnable_1 | RunnableParallel(
#     {"mul_two": runnable_2, "mul_three": runnable_3}
# )
# Also equivalently:
# sequence = runnable_1 | RunnableParallel(
#     mul_two=runnable_2,
#     mul_three=runnable_3,
# )


print(sequence.invoke(1))
# > {'mul_two': 4, 'mul_three': 6}
print(sequence.batch([1, 2, 3]))
# > [{'mul_two': 4, 'mul_three': 6}, {'mul_two': 6, 'mul_three': 9}, {'mul_two': 8, 'mul_three': 12}]


async def async_invoke(sequence, x):
   return await sequence.ainvoke(x)


async def async_batch(sequence, x):
   return await sequence.abatch(x)


{'mul_two': 4, 'mul_three': 6}
[{'mul_two': 4, 'mul_three': 6}, {'mul_two': 6, 'mul_three': 9}, {'mul_two': 8, 'mul_three': 12}]


## RunnablePassthrough
This is a runnable that passes inputs through unchanged or with additional keys. It behaves almost like the identity function, except that it can be configured to add additional keys to the output, if the input is a dictionary. 

It’s often used in conjunction with `RunnableParallel` to pass data through to a new key in the map, which allows you to keep the original input intact while adding some extra information.

In [ ]:
# %pip install -qU langchain langchain-openai

import os
from getpass import getpass

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

os.environ["OPENAI_API_KEY"] = getpass()

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

Here, the `passed` key was called with `RunnablePassthrough`, passing on the input data {'num': 1} without changes. And, the modified key was set using a lambda that added 1 to 'num', resulting in `modified` having the value `2`.

## RunnableLambda
`RunnableLambda` is a LangChain abstraction that allows Python-callable functions to be transformed into functions compatible with LangChain's pipeline operations. 

Wrapping a callable in a `RunnableLambda` makes the callable usable within either a `sync` or `async` context and can be composed as any other runnable.

In [8]:
# This is a RunnableLambda
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

runnable = RunnableLambda(add_one)

runnable.invoke(1) # returns 2
runnable.batch([1, 2, 3]) # returns [2, 3, 4]

# Async is supported by default by delegating to the sync implementation
await runnable.ainvoke(1) # returns 2
await runnable.abatch([1, 2, 3]) # returns [2, 3, 4]


# Alternatively, can provide both synd and sync implementations
async def add_one_async(x: int) -> int:
    return x + 1

runnable = RunnableLambda(add_one, afunc=add_one_async)
runnable.invoke(1) # Uses add_one
await runnable.ainvoke(1) # Uses add_one_async

2

As shown above, the code handles individual values and batches of data, using the provided `sync` and `async` implementations. 